In [10]:
# %%capture
!pip install gym seaborn keras-rl2 mlflow[extras]

     -------------------------------------- 293.3/293.3 KB 1.8 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\meris\Documents\RA_work_DRMarzieh\new-files\rl-epidemic-control\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
from episode_runner import ep_run

ImportError: cannot import name 'LegacyVersion' from 'packaging.version' (c:\Users\meris\Documents\RA_work_DRMarzieh\new-files\rl-epidemic-control\venv\lib\site-packages\packaging\version.py)

In [36]:
# training
reward, dqn = ep_run(plot_title="Training", train=True).run_experimentation(run_name = "rl-training", lr=1e-3, num_steps=20000)

In [ ]:

# testing
test_ep_runner = ep_run(action_space=Discrete(1), plot_title="continuous restrictions", train=False) # lockdown
test_reward = test_ep_runner.test(test_ep_runner.env, experiment_type="lockdown", num_episodes=1)